# VQE Prototype
A simple example for a VQE utilizing the full qiskit library and a simple test hamiltonian following <a href = https://github.com/DavitKhach/quantum-algorithms-tutorials/blob/master/variational_quantum_eigensolver.ipynb>this tutorial</a>.
. The Hamiltonian will be described as the Pauli string
$$H = 0.6 * IX + 0.8 * IZ + 0.2 * XY $$
where the multiplication without any symol in between denotes the Kronecker Product.

System Settings: Since Qiskit Aqua has been deprecated, this example may not run on the newest versions of python and qiskit. This example was created using python 3.9 and qiskit 0.31. Newer versions may not be compatible.

In [31]:
# Imports
import numpy as np
from random import random
from scipy.optimize import minimize


from qiskit import *
from qiskit.circuit.library.standard_gates import U2Gate
from qiskit.aqua.operators import WeightedPauliOperator
from qiskit.aqua.algorithms import NumPyEigensolver

In [32]:
def hamiltonian_operator(a, b, c, d):
    """
    Creates a*I + b*Z + c*X + d*Y pauli sum
    that will be our Hamiltonian operator.

    """
    pauli_dict = {
        'paulis': [{"coeff": {"imag": 0.0, "real": a}, "label": "I"},
                   {"coeff": {"imag": 0.0, "real": b}, "label": "Z"},
                   {"coeff": {"imag": 0.0, "real": c}, "label": "X"},
                   {"coeff": {"imag": 0.0, "real": d}, "label": "Y"}
                   ]
    }
    return WeightedPauliOperator.from_dict(pauli_dict)

In [33]:
# coefficient definition
scale = 10
a, b, c, d = (scale*random(), scale*random(),
              scale*random(), scale*random())
H = hamiltonian_operator(a, b, c, d)


In [34]:
#confirm that H is indeed acting on two qubits
print(H)

Representation: paulis, qubits: 1, size: 4


In [35]:
# use numpy eigensolver to exactly determine the eigenstates of the hamiltonion
# note that there is no quantum speedup in this
exact_result = NumPyEigensolver(H).run()
reference_energy = min(np.real(exact_result.eigenvalues))
print('The exact ground state energy is: {}'.format(reference_energy))

The exact ground state energy is: -1.9525997152936638


The energy is negative as it is a bound energy state.

## Creating the Ansatz state:
As Ansatz state, we start with the method proposed by the tutorial, i.e. in performing two rotations Rx, Ry onto each of our qubits, specified by parameters t1 - t4. It can easily be shown on the Bloch sphere that two rotations about different axes are universal i.e. each possible state on the Bloch sphere can be reached.

Later this will be changed in favour of the k-UCC Ansatz state.

In [36]:
# creates a circuit doing the preparation on an initial qubit in the |0> state
def quantum_state_preparation(circuit, parameters):
    q = circuit.qregs[0] # q is the quantum register where the info about qubits is stored
    circuit.rx(parameters[0], q[0]) # q[0] is our one and only qubit XD
    circuit.ry(parameters[1], q[0])
    return circuit


In [37]:
H_gate = U2Gate(0, np.pi).to_matrix()
print("H_gate:")
print((H_gate * np.sqrt(2)).round(5))

Y_gate = U2Gate(0, np.pi/2).to_matrix()
print("Y_gate:")
print((Y_gate * np.sqrt(2)).round(5))

H_gate:
[[ 1.+0.j  1.-0.j]
 [ 1.+0.j -1.+0.j]]
Y_gate:
[[ 1.+0.j -0.-1.j]
 [ 1.+0.j  0.+1.j]]


## Creating the circuit

In [38]:


def vqe_circuit(parameters, measure):
    """
    Creates a device ansatz circuit for optimization.
    :param parameters_array: list of parameters for constructing ansatz state that should be optimized.
    :param measure: measurement type. E.g. 'Z' stands for Z measurement.
    :return: quantum circuit.
    """
    q = QuantumRegister(1)
    c = ClassicalRegister(1)
    circuit = QuantumCircuit(q, c)

    # quantum state preparation
    circuit = quantum_state_preparation(circuit, parameters)

    # measurement
    if measure == 'Z':
        circuit.measure(q[0], c[0])
    elif measure == 'X':
        circuit.u2(0, np.pi, q[0])
        circuit.measure(q[0], c[0])
    elif measure == 'Y':
        circuit.u2(0, np.pi/2, q[0])
        circuit.measure(q[0], c[0])
    else:
        raise ValueError('Not valid input for measurement: input should be "X" or "Y" or "Z"')

    return circuit



In [39]:
# quantum module is the function governing the vqe measurements on top of the circuit creation
def quantum_module(parameters, measure):
    # measure
    if measure == 'I':
        return 1
    elif measure == 'Z':
        circuit = vqe_circuit(parameters, 'Z')
    elif measure == 'X':
        circuit = vqe_circuit(parameters, 'X')
    elif measure == 'Y':
        circuit = vqe_circuit(parameters, 'Y')
    else:
        raise ValueError('Not valid input for measurement: input should be "I" or "X" or "Z" or "Y"')

    shots = 8192
    backend = BasicAer.get_backend('qasm_simulator')
    job = execute(circuit, backend, shots=shots)
    result = job.result()
    counts = result.get_counts()

    # expectation value estimation from counts
    expectation_value = 0
    for measure_result in counts:
        sign = +1
        if measure_result == '1':
            sign = -1
        expectation_value += sign * counts[measure_result] / shots

    return expectation_value


In [40]:
def pauli_operator_to_dict(pauli_operator):
    """
    from WeightedPauliOperator return a dict:
    {I: 0.7, X: 0.6, Z: 0.1, Y: 0.5}.
    :param pauli_operator: qiskit's WeightedPauliOperator
    :return: a dict in the desired form.
    """
    d = pauli_operator.to_dict()
    paulis = d['paulis']
    paulis_dict = {}

    for x in paulis:
        label = x['label']
        coeff = x['coeff']['real']
        paulis_dict[label] = coeff

    return paulis_dict
pauli_dict = pauli_operator_to_dict(H)

In [41]:
# confirm that everything went right
print(pauli_dict)

{'I': 5.848267121885352, 'Z': 4.132729422345979, 'X': 4.723944151599246, 'Y': 4.632323670216331}


In [42]:
def vqe(parameters):

    # quantum_modules
    quantum_module_I = pauli_dict['I'] * quantum_module(parameters, 'I')
    quantum_module_Z = pauli_dict['Z'] * quantum_module(parameters, 'Z')
    quantum_module_X = pauli_dict['X'] * quantum_module(parameters, 'X')
    quantum_module_Y = pauli_dict['Y'] * quantum_module(parameters, 'Y')

    # summing the measurement results
    classical_adder = quantum_module_I + quantum_module_Z + quantum_module_X + quantum_module_Y

    return classical_adder

In [43]:

parameters_array = np.array([np.pi, np.pi, np.pi, np.pi])
tol = 1e-3 # tolerance for optimization precision.

vqe_result = minimize(vqe, parameters_array, method="Powell", tol=tol)
print('The exact ground state energy is: {}'.format(reference_energy))
print('The estimated ground state energy from VQE algorithm is: {}'.format(vqe_result.fun))



/tmp/ipykernel_7676/3649201999.py:19: DeprecationWarning: The QuantumCircuit.u2 method is deprecated as of 0.16.0. It will be removed no earlier than 3 months after the release date. You can use the general 1-qubit gate QuantumCircuit.u instead: u2(φ,λ) = u(π/2, φ, λ). Alternatively, you can decompose it interms of QuantumCircuit.p and QuantumCircuit.sx: u2(φ,λ) = p(π/2+φ) sx p(λ-π/2) (1 pulse on hardware).
  circuit.u2(0, np.pi, q[0])


The exact ground state energy is: -1.9525997152936638
The estimated ground state energy from VQE algorithm is: -1.9969850173318227
